In [ ]:
import numpy as np
import netCDF4 as nc

# first file
f0 = '/srv/scratch/z3533156/26year_BRAN2020/outer_avg_01461.nc'
with nc.Dataset(f0) as ds:
    temp = load_transform(ds.variables['temp'])
    sum_all = np.nansum(temp, axis=3)                      # (x,y,z)
    cnt_all = np.sum(~np.isnan(temp), axis=3).astype(np.int64)

# remaining files
fnumbers = [f'{num:05}' for num in range(1491, 10611+1, 30)]
for fnum in fnumbers:
    fname = f'/srv/scratch/z3533156/26year_BRAN2020/outer_avg_{fnum}.nc'
    with nc.Dataset(fname) as ds:
        temp = ds.variables['temp']
        temp = np.transpose(temp, axes=(3, 2, 1, 0))
        temp = np.flip(temp, axis=2)

        sum_all += np.nansum(temp, axis=3)
        cnt_all += np.sum(~np.isnan(temp), axis=3)
    print(fnum)

# climatological mean over all timesteps of all files
temp_ave = sum_all / cnt_all
temp_ave[np.isnan(temp_ave)] = np.nan  # keep NaNs where no data

np.save('/srv/scratch/z5297792/Climatology/temp_ave_not_interpolated.npy', temp_ave)
